## Note

I have discovered that kaggle has new data set but using older data for evaluation

Thus, the competition scores wrongly.

That's why I am using old version data for training.

### discussion about this issue

https://www.kaggle.com/c/dont-overfit-ii/discussion/169948

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
from scipy.stats import ks_2samp

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#train = pd.read_csv('../input/dont-overfit-ii/train.csv', index_col = 'id') # new data
#test = pd.read_csv('../input/dont-overfit-ii/test.csv', index_col = 'id') # new data

train = pd.read_csv('../input/older-dataset-for-dont-overfit-ii-challenge/train.csv', index_col = 'id') # old data
test = pd.read_csv('../input/older-dataset-for-dont-overfit-ii-challenge/test.csv', index_col = 'id') # old data
train.head(4)

In [ ]:
train_y = train['target']
train = train.drop(columns='target')
train.head(4)


In [ ]:
plt.bar(range(2), (train.shape[0], test.shape[0]), align='center', alpha=0.8)
plt.xticks(range(2), ('train','test'))
plt.ylabel('Number of data') 
plt.title('Can we avoid overfitting')
plt.show()


In [ ]:
plt.figure(figsize=(15,15))
for i in range(5):
    for j in range(5):
        plt.subplot(5,5,5*i+j+1)
        plt.hist(test[str(5*i+j)],bins=100)
        plt.title('Column '+str(5*i+j))
plt.show()


In [ ]:
# You could see all caraible columns has gaussian distribution
# with mean 0 and std 1 
# But will there any differnece with test data?

print(train.mean().sum()/300)
print(train.std().sum()/300)


In [ ]:
#from kernal  "https://www.kaggle.com/nanomathias/distribution-of-test-vs-training-data"

def get_diff_columns(train_df, test_df, show_plots=True, show_all=False, threshold=0.1):
    """Use KS to estimate columns where distributions differ a lot from each other"""

    # Find the columns where the distributions are very different
    diff_data = []
    for col in tqdm(train_df.columns):
        statistic, pvalue = ks_2samp(
            train_df[col].values, 
            test_df[col].values
        )
        if pvalue > 0.05 and np.abs(statistic) < threshold:
            diff_data.append({'feature': col, 'p': np.round(pvalue, 5), 'statistic': np.round(np.abs(statistic), 2)})

    # Put the differences into a dataframe
    diff_df = pd.DataFrame(diff_data).sort_values(by='statistic', ascending=False)
    print(f"number of features with diff distribution : {len(diff_df)}")
    if show_plots:
        # Let us see the distributions of these columns to confirm they are indeed different
        n_cols = 5
        n_rows = 5
        _, axes = plt.subplots(n_rows, n_cols, figsize=(20, 3*n_rows))
        axes = [x for l in axes for x in l]

        # Create plots
        for i, (_, row) in enumerate(diff_df.iterrows()):
            if i >= len(axes):
                break
            extreme = np.max(np.abs(train_df[row.feature].tolist() + test_df[row.feature].tolist()))
            train_df.loc[:, row.feature].apply(np.log1p).hist(
                ax=axes[i], alpha=0.5, label='Train', density=True,
                bins=np.arange(-extreme, extreme, 0.25)
            )
            test_df.loc[:, row.feature].apply(np.log1p).hist(
                ax=axes[i], alpha=0.5, label='Test', density=True,
                bins=np.arange(-extreme, extreme, 0.25)
            )
            axes[i].set_title(f"Statistic = {row.statistic}, p = {row.p}")
            axes[i].set_xlabel(f'Log({row.feature})')
            axes[i].legend()

        plt.tight_layout()
        plt.show()
        
    return diff_df

# Get the columns which differ a lot between test and train
diff_df = get_diff_columns(train, test)


In [ ]:
# As distribution of train and test are different, we use roboust scaler for scaling

data = RobustScaler().fit_transform(np.concatenate((train, test), axis=0))
train = data[:250]
train += np.random.normal(0, 0.01, train.shape)
test = data[250:]

In [ ]:
clf = LogisticRegression(class_weight='balanced', solver='liblinear', penalty ='l1', C= 0.1, max_iter=10000)
clf.fit(train, train_y)
print(f'5-fold val score : {cross_val_score(clf, train, train_y, cv=5)}')

In [ ]:
clf.fit(train, train_y)
ans = clf.predict_proba(test)
ans

In [ ]:
submit = pd.read_csv('../input/dont-overfit-ii/sample_submission.csv')
submit['target'] = ans[:,1]
submit.to_csv('submit.csv', index = False)